<h1 style="text-align : center"> FIXED POINT PATCH CODE REVIEW </h1>

# Table of Contents
* [Introduction](#Introduction)
* [Review](#Review)
	* [/include/clang/Basic/TokenKinds.def](#/include/clang/Basic/TokenKinds.def)
	* [/include/clang/Sema/DeclSpec.h](#/include/clang/Sema/DeclSpec.h)
	* [/lib/Parse/ParseDecl.cpp](#/lib/Parse/ParseDecl.cpp)
	* [/lib/Sema/SemaType.cpp](#/lib/Sema/SemaType.cpp)
	* [/include/clang/AST/ASTContext.h](#/include/clang/AST/ASTContext.h)
	* [/include/clang/AST/Expr.cpp](#/include/clang/AST/Expr.cpp)
	* [/lib/Sema/SemaExpr.cpp](#/lib/Sema/SemaExpr.cpp)


# Introduction

1. This patch includes the diffs for the lexer, parser, sema, and conversion to bitcode (clang/lib/CodeGen).
2. The diffs are against the **LLVM 3.7**.
3. The patch **doesn’t support unsigned nor unsaturated _Fract or _Accum types**

# Review

## /include/clang/Basic/TokenKinds.def

In this file we added the definition of the token that the lexer will find inside the source code:
```C++
KEYWORD(__int96    , KEYALL)
KEYWORD(_Sat         , KEYALL)   // As specified by the ISO/IEC TR 18037
KEYWORD(_Fract      , KEYALL)   // As specified by the ISO/IEC TR 18037
KEYWORD(_Accum   , KEYALL)   // As specified by the ISO/IEC TR 18037
```

## /include/clang/Sema/DeclSpec.h

Added the new Type Specifier Type for the new type. 
The [TST](http://clang.llvm.org/doxygen/namespaceclang.html#a033691a5f00979c1a22d8aa114d07e13) is used to just specifies the type using an enum.
```C++
static const TST TST_fract = clang::TST_fract;
static const TST TST_accum = clang::TST_accum;
static const TST TST_int96 = clang::TST_int96;
```

Addded the function **setSaturated** in order to memorize this information while parsing the declaration
The function setSaturated is used inside **Parser::ParseDeclarationSpecifiers** when meet a kw__Sat.

Saturated is a global variable inside this file that indicates if a declaration is a Sat or not
```C++
void setSaturated(SourceLocation l) { Saturated = true; SaturatedLoc = l; }
bool isSaturated() const { return Saturated; }
```

## /lib/Parse/ParseDecl.cpp

in the function **Parser::ParseDeclarationSpecifiers**  ( row 2618 ) added the cases in order to handle the new type. For example in the case of the kw__Sat we are going to mark as Saturated the DS, using the function previously defined setSaturated().
```C++
ParseGNUAttributes(DS.getAttributes(), nullptr, LateAttrs);
continue;
  
+ //<Synopsys> FixedPoint
+     case tok::kw__Sat:
+       if (DS.isSaturated()) Diag(Tok, diag::err_sat_specified);
+       DS.setSaturated(Tok.getLocation());
+       break;
+     case tok::kw__Fract:
+       isInvalid = DS.SetTypeSpecType(DeclSpec::TST_fract, Loc, PrevSpec,
+                                                  DiagID, Policy);
+       break;
+     case tok::kw__Accum:
+       isInvalid = DS.SetTypeSpecType(DeclSpec::TST_accum, Loc, PrevSpec,
+                                                              DiagID, Policy);
+       break;
+     case tok::kw___int96:
+       isInvalid = DS.SetTypeSpecType(DeclSpec::TST_int96, Loc, PrevSpec,
+                                                                DiagID, Policy);
+       break;
+ //</Synopsys> FixedPoint
+ 
// Microsoft declspec support.
case tok::kw___declspec:
ParseMicrosoftDeclSpecs(DS.getAttributes());
```

Added in the function **Parser::isKnownToBeTypeSpecifier** the cases in order to understand if we have a known type specifier.
```C++
    case tok::kw__Decimal64:
    case tok::kw__Decimal128:
    case tok::kw___vector:
+ //<Synopsys> FixedPoint
+   case tok::kw__Sat:
+   case tok::kw__Fract:
+   case tok::kw__Accum:
+   case tok::kw___int96:
+ //</Synopsys> FixedPoint
```

## /lib/Sema/SemaType.cpp

Here too, added the cases for handling the new TST and fill correctly the Declaration Specifier ( DS ).

A [Declaration Specifier](http://clang.llvm.org/doxygen/DeclSpec_8h_source.html) encompasses storage-class-specifiers, type-specifiers, type-qualifiers, and function-specifiers. It basically describes the content of a declaration.

## /include/clang/AST/ASTContext.h

added the new CanQualType in order to handle the new fixed point type and the respective getter.

A CanQualType represents a canonical, potentially-qualified type. The CanQual template is a lightweight smart pointer that provides access to the canonical representation of a type, where all typedefs and other syntactic sugar has been eliminated. A CanQualType may also have various qualifiers (const, volatile, restrict) attached to it.

## /include/clang/AST/Expr.cpp

Implementation of the function **commuteFixedPoint()**,
Basically if the type of the operand involved in the expression is Fract or Accum, if we match one of the operation in the switch then we are going to swap the SubExprs[0] and SubExprs[1].
( What is SubExprs? For what reasons the swap? )

## /lib/Sema/SemaExpr.cpp

This file implements semantic analysis for expressions.

Added inside the function **handleFloatConversion()**
```C++
! //<Synopsys> FixedPoint
!   if (LHSType->isFractOrAccumType() && RHSType->isIntegerType()) {
!     QualType T = S.Context.getMaximalAccumType(LHSType);
!     RHS = S.ImpCastExprToType(RHS.get(), T, CK_FloatingCast);
!     RHSType = T;
!   }
!   else if (RHSType->isFractOrAccumType() && LHSType->isIntegerType()) {
!     QualType T = S.Context.getMaximalAccumType(RHSType);
!     LHS = S.ImpCastExprToType(LHS.get(), T, CK_FloatingCast);
!     LHSType = T;
!   }
! //</Synopsys> FixedPoint
! 
!   bool LHSFloat = LHSType->isRealOrFixed();
!   bool RHSFloat = RHSType->isRealOrFixed();
  
    // If we have two real floating types, convert the smaller operand
    // to the bigger result.
    if (LHSFloat && RHSFloat) {
      int order = S.Context.getFloatingTypeOrder(LHSType, RHSType);
+     if (order == 0) // Seen when error nodes in tree  -- SYNOPSYS
+       return LHSType;
      if (order > 0) {
        RHS = S.ImpCastExprToType(RHS.get(), LHSType, CK_FloatingCast);
        return LHSType;
```

HandleFloatConversion basically introduces the Expression in order to perform the casts
```C++
+ bool Type::isRealOrFixed() const {
+   return isRealFloatingType() || isFractOrAccumType();
+ }
```